In [2]:
import edhec_risk_kit_206 as erk
import numpy as np
import pandas as pd


ind49_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2013":"2018"][['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']]
ind49_mcap = erk.get_ind_market_caps(49, weights=True)["2013":"2018"][['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']]
ind49_mcap.tail()

,Hlth,Fin,Whlsl,Rtail,Food
2018-08,0.005456,0.024948,0.010987,0.076225,0.013758
2018-09,0.005577,0.024080,0.010992,0.080234,0.013394
2018-10,0.005383,0.023262,0.010681,0.080886,0.012858
2018-11,0.005370,0.024093,0.010674,0.078849,0.014076
2018-12,0.005481,0.023729,0.010849,0.077102,0.013871


In [5]:
ind49_mcap = erk.get_ind_market_caps(49, weights=True)["2013":]
inds = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']
w_eq_ = ind49_mcap[inds].iloc[0]
w_eq_ = w_eq_/w_eq_.sum()
w_eq_

Hlth     0.041663
Fin      0.175362
Whlsl    0.097411
Rtail    0.546388
Food     0.139176
Name: 2013-01, dtype: float64

In [10]:
def implied_returns(delta, sigma, w):
    """
Obtain the implied expected returns by reverse engineering the weights
Inputs:
delta: Risk Aversion Coefficient (scalar)
sigma: Variance-Covariance Matrix (N x N) as DataFrame
    w: Portfolio weights (N x 1) as Series
Returns an N x 1 vector of Returns as Series
    """
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

In [13]:
ind_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2013":"2018"][['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']]
volsM = pd.DataFrame(ind_rets.std(),index=['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food'], columns=["vol"]) 
vols = volsM*np.sqrt(12)
rho = ind_rets.corr()

VOL = vols['vol']
sigma_prior_ =VOL.dot(VOL.T) * rho
pi = implied_returns(delta=2.5, sigma=sigma_prior_, w = w_eq_)
pi

Hlth     0.152910
Fin      0.175580
Whlsl    0.201836
Rtail    0.224827
Food     0.158115
Name: Implied Returns, dtype: float64

In [14]:
# Germany will outperform other European Equities (i.e. FR and UK) by 5%
# Hlth will outperform Rtail and Whlsl by 3%
q = pd.Series([.03]) # just one view
# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*len(inds), index=inds).T
# find the relative market caps of FR and UK to split the
# relative outperformance of DE ...
w_Rtail =  w_eq_.loc["Rtail"]/(w_eq_.loc["Rtail"]+w_eq_.loc["Whlsl"])
w_Whlsl =  w_eq_.loc["Whlsl"]/(w_eq_.loc["Whlsl"]+w_eq_.loc["Rtail"])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Rtail'] = -w_Rtail
p.iloc[0]['Whlsl'] = -w_Whlsl
#(p*100).round(1)
p

,Hlth,Fin,Whlsl,Rtail,Food
0,1.0,0.0,-0.151306,-0.848694,0.0


In [17]:
from numpy.linalg import inv

# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N Covariance Matrix as DataFrame
    tau: a scalar
    p: a K x N DataFrame linking Q and Assets
    returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)

def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)

In [18]:
delta = 2.5
tau = 0.05 # from Footnote 8
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_eq_, sigma_prior_, p, q, tau = tau)
#(bl_mu*100).round(1)
bl_mu

Hlth     0.179643
Fin      0.169252
Whlsl    0.193340
Rtail    0.199847
Food     0.145319
dtype: float64

In [20]:
def inverse(d):
    """
    Invert the dataframe by inverting the underlying matrix
    """
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def w_star(delta, sigma, mu):
    return (inverse(sigma).dot(mu))/delta

wstar = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
# display w*
#(wstar*100).round(1)
wstar

Hlth     0.263700
Fin      0.167011
Whlsl    0.058876
Rtail    0.330244
Food     0.132549
dtype: float64

In [21]:
# Germany will outperform other European Equities (i.e. FR and UK) by 5%
# Hlth will outperform Rtail and Whlsl by 3%
q = pd.Series([.05]) # just one view
delta = 2.5
tau = 0.05 # from Footnote 8
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_eq_, sigma_prior_, p, q, tau = tau)
#(bl_mu*100).round(1)
bl_mu

Hlth     0.185075
Fin      0.167967
Whlsl    0.191614
Rtail    0.194772
Food     0.142719
dtype: float64

In [22]:
wstar = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
# display w*
#(wstar*100).round(1)
wstar

Hlth     0.310774
Fin      0.167011
Whlsl    0.051754
Rtail    0.290293
Food     0.132549
dtype: float64